# Module 2: Your First Neural Network with `nn.Module`

---
## What Is a Neural Network, Really?

### 🧠 Brain Analogy
A neural network is a **simplified copy of how your brain works**. Your brain has billions of neurons. Each receives signals, checks "is this strong enough to pass on?", then fires or stays silent. Signals flow layer by layer: eyes → visual cortex → recognition → decision.

PyTorch does the same with numbers:
- **Neurons** = numbers (activations)
- **Connections** = weights — how strongly one neuron influences the next
- **"Strong enough?"** = activation function (ReLU) — the firing threshold
- **Learning from mistakes** = backpropagation + optimizer

### ⚙️ Engineer Analogy
Each layer is: `output = activation(W @ input + b)`. Stack enough layers and the universal approximation theorem says you can approximate ANY function. Each layer extracts increasingly abstract features.

**Level:** Beginner  
**Duration:** ~2.5 hours  
**Dataset:** Iris Flower Dataset ([Kaggle](https://www.kaggle.com/datasets/uciml/iris))  
**Real-World Use Case:** Multi-class species classification from measurements

## What You'll Learn
- Building models with `nn.Module` and `nn.Sequential`
- Activation functions: ReLU, Sigmoid, Softmax
- Loss functions: CrossEntropyLoss, MSELoss
- Optimizers: SGD, Adam
- The standard PyTorch training loop pattern
- Evaluating classification accuracy

## The PyTorch Training Loop — Memorize This Pattern!

```
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()     ← 1. Clear old gradients
    predictions = model(X)    ← 2. Forward pass
    loss = criterion(pred, y) ← 3. Compute loss
    loss.backward()           ← 4. Backprop
    optimizer.step()          ← 5. Update weights
```

In [ ]:
# 🧠 Gathering all tools — like laying out textbooks before studying.
# ⚙️ torch.nn = layer library | torch.optim = weight-update algorithms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 2.1  Load & Explore the Iris Dataset

### 🧠 Brain Analogy
You're a botanist measuring 4 things about each flower. From just those 4 numbers, your brain must decide: which of 3 species is this? This is classification — the brain does it constantly ("dog or cat?").

### ⚙️ Engineer Analogy
Multi-class classification: input `x ∈ ℝ⁴` → label `y ∈ {0,1,2}`. 150 labelled examples are the "worked problems" the model studies from.

150 flowers × 4 features → 3 species  
*Same dataset available at* [kaggle.com/datasets/uciml/iris](https://www.kaggle.com/datasets/uciml/iris)

In [ ]:
# 🧠 Opening a textbook of 150 worked examples: measurements → correct species answer
# ⚙️ X_raw: (150,4) feature matrix | y_raw: integer class labels 0/1/2
iris = load_iris()
X_raw = iris.data           # (150, 4)  features
y_raw = iris.target         # (150,)    labels 0/1/2
class_names = iris.target_names
feature_names = iris.feature_names

df = pd.DataFrame(X_raw, columns=feature_names)
df['species'] = [class_names[i] for i in y_raw]

print("Shape:", df.shape)
print(df.head())
print("\nClass distribution:\n", df['species'].value_counts())

In [ ]:
# 🧠 Flip through the textbook before studying — do the measurements visually separate species?
# ⚙️ EDA: overlapping histograms = hard problem; separated = easy. Petals are most discriminative.
# ── Visualise feature distributions ─────────────────────────────────────
fig, axes = plt.subplots(2, 2, figsize=(10, 7))
colors = ['#2196F3', '#4CAF50', '#FF9800']

for ax, feat in zip(axes.flatten(), feature_names):
    for i, (name, color) in enumerate(zip(class_names, colors)):
        ax.hist(X_raw[y_raw == i, feature_names.index(feat)],
                bins=15, alpha=0.6, color=color, label=name)
    ax.set_title(feat)
    ax.legend(fontsize=8)

plt.suptitle("Iris Feature Distributions by Species", fontsize=13)
plt.tight_layout()
plt.show()

## 2.2  Data Preparation

### 🧠 Brain Analogy
Standardise units (all measurements on same scale), split into study pile (80%) and mock exam (20%), study in shuffled batches of 16 — prevents memorising card order.

### ⚙️ Engineer Analogy
`StandardScaler`: z-score normalisation, fit ONLY on train (no data leakage). DataLoader: mini-batches, shuffle each epoch. stratify: equal class proportions in both splits.



In [ ]:
# 🧠 Split cards 80/20, shuffle, convert to brain format (tensors). Standardise to same scale.
# ⚙️ Fit scaler on train only — same μ,σ applied to test to prevent leakage
# ── Train/test split ─────────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# ── Standardize features (zero mean, unit variance) ──────────────────────
scaler  = StandardScaler()
X_train = scaler.fit_transform(X_train)   # fit only on train!
X_test  = scaler.transform(X_test)

# ── Convert to PyTorch tensors ───────────────────────────────────────────
X_train_t = torch.FloatTensor(X_train).to(device)
y_train_t = torch.LongTensor(y_train).to(device)
X_test_t  = torch.FloatTensor(X_test).to(device)
y_test_t  = torch.LongTensor(y_test).to(device)

# ── Wrap in DataLoader for mini-batch training ───────────────────────────
train_ds     = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)

print(f"Train: {X_train_t.shape}   Test: {X_test_t.shape}")

## 2.3  Building the Model — Three Approaches

### 🧠 Brain Analogy
Each `nn.Linear(in, out)` is a group of neurons where every input connects to every output. `ReLU` is the firing threshold: "pass signal only if positive." `Dropout` randomly silences 20% of neurons during training, forcing the brain to build redundant paths.

### ⚙️ Engineer Analogy
`nn.Sequential` = Unix pipe. Custom `nn.Module` = circuit board with explicit wiring. Required for skip connections, branching, shared weights. `Dropout(p=0.2)` = Bernoulli noise regularisation, active only in `train()` mode.

### Approach A: `nn.Sequential` (quick, readable)

In [ ]:
# 🧠 Assembly line: 4 measurements → layer-by-layer transformation → 3 species scores
# ⚙️ No softmax at end — CrossEntropyLoss applies log-softmax internally (more stable)
model_seq = nn.Sequential(
    nn.Linear(4, 16),       # input: 4 features → 16 hidden neurons
    nn.ReLU(),
    nn.Linear(16, 8),       # 16 → 8
    nn.ReLU(),
    nn.Linear(8, 3)         # 8 → 3 classes (raw logits, no softmax)
).to(device)

print(model_seq)
print(f"\nTotal parameters: {sum(p.numel() for p in model_seq.parameters()):,}")

### Approach B: Custom `nn.Module` class (preferred for complex models)

### 🧠 Brain Analogy
`nn.Module` is the **brain circuit blueprint**. `__init__` lists components (layers); `forward(x)` wires them. `Dropout(p=0.2)` randomly silences 20% — forces robust redundant representations.

### ⚙️ Engineer Analogy
`nn.Module` auto-registers parameters, enables `train()`/`eval()` switching, provides `state_dict()`. Dropout = stochastic regularisation — active in `train()`, no-op in `eval()`.



In [ ]:
# 🧠 Blueprint: 4 senses → 16 detectors → 8 patterns → 3 species decisions
# ⚙️ __init__: declare components | forward: define signal flow (the wiring diagram)
class IrisNet(nn.Module):
    """Feed-forward network for Iris species classification."""

    def __init__(self, input_dim=4, hidden_dim=16, num_classes=3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim,  hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, num_classes)
        self.relu    = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)      # regularization

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)                       # raw logits out
        return x


model = IrisNet().to(device)
print(model)

# Test with a dummy batch
dummy = torch.randn(8, 4).to(device)
out   = model(dummy)
print(f"\nOutput shape for batch of 8: {out.shape}")  # (8, 3)

## 2.4  Loss Function & Optimizer

### 🧠 Brain Analogy
**CrossEntropyLoss** = "How confident were you in the WRONG answer?" (big confidence + wrong = big regret). **Adam** = intelligent study strategy — remembers gradient history, adapts step size per weight. **ReduceLROnPlateau** = "slow down when you're not improving."

### ⚙️ Engineer Analogy
`CrossEntropyLoss` = `-log(p_correct)`, takes raw logits NOT softmax output. Adam maintains momentum + variance per parameter — adapts per-weight LR. `ReduceLROnPlateau` halves LR after `patience` epochs of stagnation.



In [ ]:
# 🧠 Loss = how wrong is the guess? Optimizer = how to fix it? Scheduler = when to slow down?
# ⚙️ CrossEntropyLoss takes raw logits — DO NOT apply softmax before passing to it
# CrossEntropyLoss = LogSoftmax + NLLLoss  (takes raw logits, NOT probabilities)
criterion = nn.CrossEntropyLoss()

# Adam optimizer: adaptive learning rates (usually better than vanilla SGD)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Optional: learning rate scheduler (reduce lr when plateau)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)

## 2.5  Training Loop

### 🧠 Brain Analogy
Every mini-batch: (1) wipe whiteboard, (2) make guess, (3) score it, (4) trace which neurons caused error, (5) adjust those neurons. After 150 rounds of this across hundreds of mini-batches, the brain has refined its flower knowledge.

### ⚙️ Engineer Analogy
`θ ← θ - lr * Adam(∇L)`. `model.train()` enables Dropout. `model.eval()` disables it. `@torch.no_grad()` skips gradient tape construction — ~30% faster for evaluation.



In [ ]:
# 🧠 5-step ritual: wipe → guess → score → trace error → adjust. Repeat 150 rounds.
# ⚙️ zero_grad() MUST precede backward() — gradients accumulate by default
def train_epoch(model, loader, criterion, optimizer):
    model.train()           # enable dropout / batchnorm training mode
    total_loss, correct = 0.0, 0

    for X_batch, y_batch in loader:
        optimizer.zero_grad()                    # ① clear gradients
        logits = model(X_batch)                  # ② forward
        loss   = criterion(logits, y_batch)      # ③ loss
        loss.backward()                          # ④ backprop
        optimizer.step()                         # ⑤ update

        total_loss += loss.item() * len(X_batch)
        preds = logits.argmax(dim=1)
        correct += (preds == y_batch).sum().item()

    return total_loss / len(loader.dataset), correct / len(loader.dataset)


@torch.no_grad()
def evaluate(model, X, y, criterion):
    model.eval()            # disable dropout
    logits = model(X)
    loss   = criterion(logits, y).item()
    preds  = logits.argmax(dim=1)
    acc    = (preds == y).float().mean().item()
    return loss, acc


# ── Run training ─────────────────────────────────────────────────────────
EPOCHS = 150
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = train_epoch(model, train_loader, criterion, optimizer)
    vl_loss, vl_acc = evaluate(model, X_test_t, y_test_t, criterion)
    scheduler.step(vl_loss)

    history['train_loss'].append(tr_loss)
    history['train_acc'].append(tr_acc)
    history['val_loss'].append(vl_loss)
    history['val_acc'].append(vl_acc)

    if epoch % 25 == 0:
        print(f"Epoch {epoch:3d}/{EPOCHS} | "
              f"Train loss: {tr_loss:.4f} acc: {tr_acc:.3f} | "
              f"Val loss: {vl_loss:.4f} acc: {vl_acc:.3f}")

## 2.6  Visualise Training

### 🧠 Brain Analogy
Loss curve = learning diary. Rapid early drop → brain learns obvious patterns fast. Flattening = harder patterns. Big gap between train/val = "memorised cards but can't generalise" (overfitting).

### ⚙️ Engineer Analogy
Flat loss → vanishing gradients or LR too low. Oscillating → LR too high. Val loss rising while train drops → overfitting → add Dropout or reduce model.



In [ ]:
# 🧠 Left: fewer mistakes over time? Right: more correct guesses? Gap = overfitting?
# ⚙️ Healthy training: train and val curves track each other closely
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'],   label='Validation')
ax1.set_xlabel('Epoch'); ax1.set_ylabel('Loss')
ax1.set_title('Loss Curves'); ax1.legend()

ax2.plot(history['train_acc'], label='Train')
ax2.plot(history['val_acc'],   label='Validation')
ax2.set_xlabel('Epoch'); ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracy Curves'); ax2.legend()

plt.tight_layout()
plt.show()

print(f"Final Validation Accuracy: {history['val_acc'][-1]:.1%}")

## 2.7  Evaluation — Confusion Matrix

### 🧠 Brain Analogy
Confusion matrix shows WHERE the brain got confused: "when the flower was virginica, how many times did it say versicolor?" Off-diagonal = errors. Setosa is easy (very distinct petals); versicolor/virginica are similar → higher confusion.

### ⚙️ Engineer Analogy
`cm[i,j]` = count of true class `i` predicted as `j`. Precision/recall/F1 per class — more informative than raw accuracy for imbalanced problems.



In [ ]:
# 🧠 Exam time: show unseen flowers, record every answer, build the error grid
# ⚙️ model.eval() + no_grad() = inference mode; preds = argmax(logits)
model.eval()
with torch.no_grad():
    logits = model(X_test_t)
    preds  = logits.argmax(dim=1).cpu().numpy()

cm = confusion_matrix(y_test, preds)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.ylabel('True'); plt.xlabel('Predicted')
plt.title('Confusion Matrix — Iris Classification')
plt.tight_layout()
plt.show()

print(classification_report(y_test, preds, target_names=class_names))

## 2.8  Saving & Loading Models

### 🧠 Brain Analogy
Saving `state_dict` = photographing all synaptic strengths to disk. Loading = restoring the brain from that snapshot — all learned knowledge returns instantly. Need BOTH the blueprint (class definition) AND the saved weights.

### ⚙️ Engineer Analogy
`state_dict()` = ordered dict of `{layer_name: weight_tensor}`. Save state_dict, not whole model — portable across PyTorch versions. `map_location=device` handles GPU→CPU loading.



In [ ]:
# 🧠 Photograph synaptic strengths → save → restore later with all knowledge intact
# ⚙️ state_dict = lightweight dict of tensors; map_location handles device differences
# ── Save ──────────────────────────────────────────────────────────────────
# Best practice: save only the state_dict (weights), not the entire model
torch.save(model.state_dict(), 'iris_model.pth')
print("Model saved to iris_model.pth")

# ── Load ──────────────────────────────────────────────────────────────────
loaded_model = IrisNet().to(device)
loaded_model.load_state_dict(torch.load('iris_model.pth', map_location=device))
loaded_model.eval()

# Verify
with torch.no_grad():
    _, acc = evaluate(loaded_model, X_test_t, y_test_t, criterion)
print(f"Loaded model accuracy: {acc:.1%}")

## 2.9  Inference on New Samples

### 🧠 Brain Analogy
Real-world use: give the trained botanist-brain a NEW flower and ask "what species?" It returns probabilities: "94% sure virginica." Confidence = how far from 0.5.

### ⚙️ Engineer Analogy
Inference: preprocess with the SAME scaler (same μ,σ) → tensor → `eval()` forward → softmax → argmax.



In [ ]:
# 🧠 New flower → trained brain makes its best guess using all learned knowledge
# ⚙️ ALWAYS use the same scaler transform as training — wrong normalisation = wrong features
def predict_species(measurements: list) -> str:
    """Predict Iris species from [sepal_len, sepal_wid, petal_len, petal_wid]."""
    x = torch.FloatTensor(scaler.transform([measurements])).to(device)
    with torch.no_grad():
        logits = loaded_model(x)
        probs  = torch.softmax(logits, dim=1)[0]
        pred   = probs.argmax().item()
    return class_names[pred], {n: f"{p:.1%}" for n, p in zip(class_names, probs.tolist())}


# Test with known samples
samples = [
    ([5.1, 3.5, 1.4, 0.2], "setosa"),
    ([6.1, 2.8, 4.7, 1.2], "versicolor"),
    ([6.3, 3.3, 6.0, 2.5], "virginica"),
]

for features, true_label in samples:
    pred, probs = predict_species(features)
    status = '✓' if pred == true_label else '✗'
    print(f"{status} True: {true_label:12s} → Predicted: {pred:12s} | {probs}")

## 2.10  Key Concepts Summary

| Concept | PyTorch API | Notes |
|---------|------------|-------|
| Linear layer | `nn.Linear(in, out)` | Weight + bias |
| Activation | `nn.ReLU()`, `nn.Sigmoid()` | After linear |
| Dropout | `nn.Dropout(p)` | Regularization |
| Loss | `nn.CrossEntropyLoss()` | Multi-class |
| Optimizer | `optim.Adam(params, lr)` | Adaptive LR |
| Train mode | `model.train()` | Enable dropout |
| Eval mode | `model.eval()` | Disable dropout |
| No gradient | `torch.no_grad()` | Faster inference |
| Save | `torch.save(model.state_dict(), path)` | |
| Load | `model.load_state_dict(torch.load(path))` | |

## Exercises

1. Add **Batch Normalization** (`nn.BatchNorm1d`) after each hidden layer. Does it improve results?
2. Try `optim.SGD` with `momentum=0.9` vs Adam. Which converges faster?
3. Download the full Iris CSV from Kaggle and load it with `pd.read_csv` instead of sklearn.
4. Plot the **decision boundary** for the first two features using a meshgrid.

---
**Next →** [Module 03: Data Pipelines with Dataset & DataLoader (Titanic)](./Module_03_Data_Pipeline_Titanic.ipynb)